In [2]:
import torchvision.models as models
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy,copy
# the magic number
mean, std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

preprocess_augment = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

preprocess = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

from myDataset import PalmNutriDataset
ground_truth = 'dataset/gt.csv'
full_train_dataset = PalmNutriDataset(ground_truth=ground_truth, img_dir='dataset', sample_set='n17', target='n')
print(len(full_train_dataset))
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=preprocess)

train_dataset, val_dataset = torch.utils.data.random_split(full_train_dataset, [800,218])
train_dataset.dataset = copy(full_train_dataset)
train_dataset.dataset.transform = preprocess_augment
val_dataset.dataset.transform = preprocess

1018


In [3]:
BATCH_SIZE=32
NUM_WORKERS=2
train_dataloader = torch.utils.data.DataLoader(full_train_dataset, batch_size=BATCH_SIZE,shuffle=True , num_workers=NUM_WORKERS)
val_dataloader   = torch.utils.data.DataLoader(val_dataset  , batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)
# test_dataloader  = torch.utils.data.DataLoader(test_dataset , batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)

In [8]:
model = models.mobilenet_v3_small()

In [9]:
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [7]:
model.classifier[3] = torch.nn.Linear(in_features=1024,out_features=1,bias=True)

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)

In [7]:
from trainer import trainer
dataloaders = {'train': train_dataloader,'val':val_dataloader}
# Set device to GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model = models.mobilenet_v3_large()
# model.classifier[3] = torch.nn.Linear(in_features=1280,out_features=1,bias=True)
model = models.mobilenet_v3_small()
model.classifier[3] = torch.nn.Linear(in_features=1024,out_features=1,bias=True)
# Optimizer and loss function
criterion = nn.MSELoss()
params_to_update = model.parameters()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = optim.Adam(params_to_update, lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min')
t = trainer(device,criterion, optimizer,scheduler)
model = t.train(model, dataloaders, num_epochs=150, weights_name='alex_sgd_0.01')
# t.test(model,test_dataloader)

cuda:0
Epoch 0/149:LR: 0.0001
----------
train Loss: 1.2501 Acc: 0.0000
Epoch time taken:  8.571077108383179
val Loss: 0.1107 Acc: 0.0000
Epoch time taken:  10.737535238265991
Epoch 1/149:LR: 0.0001
----------
train Loss: 0.1496 Acc: 0.0000
Epoch time taken:  8.578619480133057
val Loss: 0.0974 Acc: 0.0000
Epoch time taken:  10.435415267944336
Epoch 2/149:LR: 0.0001
----------
train Loss: 0.1329 Acc: 0.0000
Epoch time taken:  7.88027286529541
val Loss: 0.1162 Acc: 0.0000
Epoch time taken:  9.781769514083862
Epoch 3/149:LR: 0.0001
----------
train Loss: 0.1174 Acc: 0.0000
Epoch time taken:  7.840801954269409
val Loss: 0.1092 Acc: 0.0000
Epoch time taken:  9.775903463363647
Epoch 4/149:LR: 0.0001
----------
train Loss: 0.1135 Acc: 0.0000
Epoch time taken:  8.135318994522095
val Loss: 0.0995 Acc: 0.0000
Epoch time taken:  9.943527698516846
Epoch 5/149:LR: 0.0001
----------
train Loss: 0.1089 Acc: 0.0000
Epoch time taken:  7.516961336135864
val Loss: 0.1134 Acc: 0.0000
Epoch time taken:  9.

KeyboardInterrupt: 